# IMEC2201 Herramientas Computacionales 
## Semana 2: Gráficas y Visualizaciones
### Clase 3: Configuración Espacio de Trabajo

Universidad de los Andes — Abril 4, 2022.

---

## TABLA DE CONTENIDO

### Sección 1: DataFrames.jl [→](#section1)
- 1.1. Cargar Librerías
- 1.2. Cargar Datos desde Excel
    - 1.2.1. Valores 'Missing'
    - 1.2.2. Filtrar Datos    
- 1.3. Extraer Año, Mes, Día
- 1.4. Reordenar Columnas
- 1.5. Datos Únicos con unique()
- 1.6. Query
- 1.7. Concatenar DataFrames
    - 1.7.1. DataFrame de Voltaje DC
    - 1.7.2. DataFrame de Corriente DC
    - 1.7.3. DataFrame con Irradiancia, Voltaje DC y Corriente DC
- 1.8. Descargar Archivo

### Sección 2: Plots.jl [→](#section2)
- 2.1. Bases
- 2.2. Gráficas con Datos de DataFrame
___

**¡Importante!** Note que la carpeta **week2** contiene dos archivos: `Project.toml` y `Manifest.toml`. Estos configuran el <font color="#FF5733">*ambiente de trabajo*</font> y facilitan el manejo de <font color="#FF5733">librerías</font>.

Para inicializar el ambiente desde Julia REPL, dentro de la carpeta **week2**, ejecute lo siguiente:

```shell
$ (v1.7) pkg > activate .
$ (week2) pkg> instantiate
```

**Nota:** Para acceder al Pkg REPL, escriba el corchete derecho `]` en el Julia REPL. Note que el REPL cambie de color de verde (Julia REPL) a azul (Pkg REPL). Una vez ejecutado el código `activate .`, el prefijo entre paréntesis indica el ambiente activo: cambia de `v1.7` a `week2`.

O, desde Jupyter Notebook, ejecute:

```julia
$ using Pkg
$ Pkg.activate()
$ Pkg.instantiate()
```

**Nota:** La activación del ambiente <font color="#FF5733">*precompila*</font> las librerías por lo que puede tardar un momento su ejecución completa.

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i>
Puede obtener más información en la documentación oficial de la librería [`Pkg.jl`](https://pkgdocs.julialang.org/v1/environments/) y en el documento dando clic [aquí](https://towardsdatascience.com/how-to-setup-project-environments-in-julia-ec8ae73afe9c).
</div>

In [ ]:
using Pkg
Pkg.activate(pwd())
Pkg.instantiate()

<a id="section1"></a>
# Sección 1: DataFrames.jl

## 1.1. Cargar Librerías

In [ ]:
using Pkg
Pkg.add("XLSX")
Pkg.add("CSV")
Pkg.add("DataFrames")

In [ ]:
Pkg.status()

## 1.2. Cargar Datos desde Excel

In [ ]:
using DataFrames, XLSX

rad = DataFrame(XLSX.readtable("./data/SD_5Min.xlsx", "Radiacion", header=true)...)

println("El DataFrame tiene $(size(df1)[1]) filas y $(size(df1)[2]) columnas.")

first(rad, 5)

In [ ]:
describe(rad)

### 1.2.1. Valores 'Missing'

In [ ]:
encabezados = names(rad)

for i in encabezados
    println(i)
end

In [ ]:
#=
Iteramos sobre las columnas del DataFrame 'rad'
para verificar la cantidad de datos faltantes
=#

for i in encabezados
    println("$i - $(sum(ismissing.(rad[:,i])))")
end

El siguiente código permite filtrar los datos faltantes y asignarles un valor de cero.

```julia
rad[:,"Meteocontrol Irrad (W/m2)"] = replace(rad[:,"Meteocontrol Irrad (W/m2)"], missing => 0)
```

Realizar de manera manual este prodecimiento para todas las columnas es tedioso y propenso a errores. ¿Cómo podríamos hacerlo con un *for loop*?

In [ ]:
# For Loop
for i in encabezados
    rad[:,i] = replace(rad[:,i], missing => 0)
end

# Verificación
for i in encabezados
    println("$i - $(sum(ismissing.(rad[:,i])))")
end

### 1.2.2. Filtrar Datos


Note que la columna **Meteocontrol Irrad** reporta valores mínimos hasta 1. Esto se desea filtrar para reducir el valor mínimo de 1 a 0 si la irradiancia es menor o igual a 1.5 W/m2.

In [ ]:
# Sin Filtro
first(rad, 5)

In [ ]:
#= Con Filtro

Tomado de: https://discourse.julialang.org/t/how-to-replace-set-change-value-in-array-dataframe-if-value-is-less-greater-than/51039/2
=#

rad[:,"Meteocontrol Irrad (W/m2)"] .= ifelse.(rad[:,"Meteocontrol Irrad (W/m2)"] .<= 1.5, 0.0, rad[:,"Meteocontrol Irrad (W/m2)"])

first(rad, 5)

## 1.3. Extraer Año, Mes, Día

El tipo de las columnas, visto desde el DataFrame de la celda, es `Any`.

In [ ]:
first(rad, 5)

In [ ]:
#=
También lo podemos saber al extraer la información
por cada columna, según StackOverflow
https://stackoverflow.com/questions/57863153/julia-how-to-obtain-the-types-of-every-column-of-a-dataframe-table
=#

eltype.(eachcol(rad))

¿Cuál es el tipo (`typeof()`) de los datos de la columna **Date**?

<font color="white">
for i in 1:5
    println("$(rad[i, "Date"]) - Tipo: $(typeof(rad[i, "Date"]))")
end
</font>

In [ ]:
???

`Dates.DateTime` es un tipo de dato especial para trabajar con fechas. Así, es más sencillo extraer, por ejemplo, el año, mes o día de una fecha específica. Más información [aquí](https://docs.julialang.org/en/v1/stdlib/Dates/).

In [ ]:
using Dates

println("Año: $(Dates.year(rad[1, "Date"]))")
println("Mes: $(Dates.month(rad[1, "Date"]))")
println("Día: $(Dates.day(rad[1, "Date"]))")

¿Y si queremos conocer la hora, minuto y segundos? ¿Por qué nos salta error para el primer dato?

In [ ]:
dato = 1
println("$(rad[dato, "Date"]) - Tipo: $(typeof(rad[dato, "Date"]))")
println("Hora: $(Dates.hour(rad[dato, "Date"]))")
println("Minuto: $(Dates.minute(rad[dato, "Date"]))")
println("Segundo: $(Dates.second(rad[dato, "Date"]))")

In [ ]:
dato = 2
println("$(rad[dato, "Date"]) - Tipo: $(typeof(rad[dato, "Date"]))")
println("Hora: $(Dates.hour(rad[dato, "Date"]))")
println("Minuto: $(Dates.minute(rad[dato, "Date"]))")
println("Segundo: $(Dates.second(rad[dato, "Date"]))")

Con estas funciones, iteremos el DataFrame `rad` para agregar las columnas de Año, Mes y Día.

In [ ]:
_año = []

for i in rad[:,"Date"]
    push!(_año, Dates.year(i))
end

rad[:, "Year"] = _año # Creamos la columna "Year" y asignamos los valores "_año"

first(rad, 5)

¿Cómo agregaría las columnas de Mes y Día?
<font color="white">
_mes = []
_dia = []

for i in rad[:,"Date"]
    push!(_mes, Dates.month(i))
    push!(_dia, Dates.day(i))
end

rad[:, "Month"] = _mes # Creamos la columna "Month" y asignamos los valores "_mes"
rad[:, "Day"] = _dia # Creamos la columna "Day" y asignamos los valores "_dia"

first(rad[:, ["Date", "Year", "Month", "Day"]], 5)
</font>

In [ ]:
???

## 1.4. Reordenar Columnas

In [ ]:
names(rad)

In [ ]:
rad = rad[:,["Date", "Year", "Month", "Day", "Lufft Irrad (W/m2)", "Meteocontrol Irrad (W/m2)"]]

first(rad, 5)

## 1.5. Datos Únicos con `unique()`

In [ ]:
unique(rad[:,"Year"])

In [ ]:
unique(sort(rad[:,"Month"]))

In [ ]:
unique(rad[:,"Day"])

## 1.6. Query

In [ ]:
#=
Solo queremos ver datos para Junio-2021
=#

rad[ (rad[:, "Year"] .== 2021) .&& (rad[:, "Month"] .== 6), :]

In [ ]:
#=
Solo queremos ver datos cuando Meteocontrol Irrad >= 400
=#

first(rad[(rad[:, "Meteocontrol Irrad (W/m2)"] .>= 400), :], 10)

In [ ]:
#=
Solo queremos ver datos cuando (Lufft Irrad - Meteocontrol Irrad) <= 100
=#

first(rad[ ((rad[:, "Lufft Irrad (W/m2)"] - rad[:, "Meteocontrol Irrad (W/m2)"]) .<= 100), :], 10)

<font color="white">
query = 
    rad[
        (rad[:, "Year"] .== 2020) .&
        ((rad[:, "Month"] .== 8) .| (rad[:, "Month"] .== 9)) .& 
        (  (rad[:, "Day"] .== 10)   .|   (rad[:, "Day"] .== 15)  ) .&
        (rad[:, "Meteocontrol Irrad (W/m2)"] .>= 400)
    , :]
first(query, 5)
</font>

In [ ]:
#=
Solo queremos ver datos cuando:
    1. Año = 2020
    2. Mes = Agosto y Septiembre
    3. Día = 10 o 15
    4. Meteocontrol Irrad >= 800
=#
???

In [ ]:
# Verifiquemos
println(unique(query[:, "Year"]))
println(unique(query[:, "Month"]))
println(unique(query[:, "Day"]))
println(minimum(query[:, "Meteocontrol Irrad (W/m2)"]))

Para el caso de datos tabulares en formato CSV, usamos la correspondiente librería (i.e., `CSV`).

In [ ]:
using CSV

file = CSV.File("./data/hc-users.csv", header=1, delim=';')
df2 = DataFrame(file)

first(df2, 5)

### 1.7. Concatenar DataFrames

### 1.7.1. DataFrame de Voltaje DC

In [ ]:
Vdc = DataFrame(XLSX.readtable("./data/SD_5Min.xlsx", "Tension_DC", header=true)...)

println("El DataFrame tiene $(size(Vdc)[1]) filas y $(size(Vdc)[2]) columnas.\n")

# For Loop
for i in names(Vdc)
    Vdc[:,i] = replace(Vdc[:,i], missing => 0)
end

# Verificación
for i in names(Vdc)
    println("$i - $(sum(ismissing.(Vdc[:,i])))")
end


first(Vdc, 5)

### 1.7.2. DataFrame de Corriente DC

Cree un DataFrame a partir de importar los datos de corriente DC (hoja `Corriente_DC` en el archivo Excel) y reemplace los datos con tipo `missing` a cero.

In [ ]:
Idc = DataFrame(XLSX.readtable("./data/SD_5Min.xlsx", "Corriente_DC", header=true)...)

println("El DataFrame tiene $(size(Idc)[1]) filas y $(size(Idc)[2]) columnas.\n")

# For Loop
for i in names(Idc)
    Idc[:,i] = replace(Idc[:,i], missing => 0)
end

# Verificación
for i in names(Idc)
    println("$i - $(sum(ismissing.(Idc[:,i])))")
end

first(Idc, 5)

### 1.7.3. DataFrame con Irradiancia, Voltaje DC y Corriente DC

In [ ]:
#=
Primero chequeamos que todos los DataFrame
tengan la misma cantidad de filas con 'nrow()'
=#
println("Filas DataFrame Radiación: $(nrow(rad))")
println("Filas DataFrame Voltaje: $(nrow(Vdc))")
println("Filas DataFrame Corriente: $(nrow(Idc))")

Para realizar la concatenación, tenemos varias opciones:

- `innerjoin`: la salida contiene filas para los valores de la clave que existen en los argumentos primero (izquierda) y segundo (derecha) para unir.

- `leftjoin`: la salida contiene filas para los valores de la clave que existen en el primer argumento (izquierda) para unir, ya sea que ese valor exista o no en el segundo argumento (derecha).

- `rightjoin`: la salida contiene filas para los valores de la clave que existen en el segundo argumento (derecha) para unir, ya sea que ese valor exista o no en el primer argumento (izquierda).

- `outerjoin`: la salida contiene filas para los valores de la clave que existen en el primer argumento (izquierda) o segundo (derecha) para unir.

Más info [aquí](https://dataframes.juliadata.org/stable/man/joins/).

In [ ]:
df = innerjoin(rad, Vdc, on = "Date")

describe(df)

Concatene los datos del DataFrame `Idc` al DataFrame `df`.

<font color="white">
df = innerjoin(df, Idc, on = "Date")
describe(df)
</font>

In [ ]:
???

## 1.8. Descargar Archivo

In [ ]:
CSV.write("./produccion_solar.csv", df, delim=';', decimal=',', header=true)

<a id="section2"></a>
# Sección 2: Plots.jl

## 2.1. Bases

Julia dispone de diversas librerías para generar visualizaciones ([aquí](https://juliapackages.com/c/graphics)). En este caso, usaremos `Plots.jl`.

In [ ]:
using Plots
plot(rand(4,4))

In [ ]:
import Plots
const plt = Plots # 'plt' es un alias de 'Plots'; equivalente a "import Plots as pl" en Python
plt.plot(rand(4,4))

In [ ]:
#=
Algunas propiedades de Plots.jl
=#

x = collect(0:π/12:4π)
y1 = [sin(i) for i in x]
y2 = cos.(x)

plot(x, y1, label="sin(x)")
scatter!(x, y2, label="cos(x)")

In [ ]:
xlabel!("Ángulo π")
ylabel!("Respuesta Unitaria")
title!("Comportamiento sin(x) y cos(x)")

In [ ]:
y3 = atan.(x)
y4 = exp.(x)

p1 = plot(x, y1, label="sin(x)", color="dodgerblue")
p2 = plot(x, y2, label="cos(x)", color="red")
p3 = plot(x, y3, label="atan(x)", color="orange")
p4 = plot(x, y4, label="exp(x)", color="green")

plot(p1, p2, p3, p4, layout=(2,2), legend=true)

## 2.2. Gráficas con Datos de  DataFrame

In [ ]:
first(df, 5) # Si queremos ver todo el DataFrame: show(df, allrows=true, allcols=true)

In [ ]:
names(df)

In [ ]:
query = 
    
    rad[
        (rad[:, "Date"] .>= DateTime("2020-01-01", dateformat"yyyy-mm-dd hh:mm:ss")) .&
        (rad[:, "Date"] .<= DateTime("2020-01-07", dateformat"yyyy-mm-dd hh:mm:ss"))
    , :]


println(first(query))
println()
println(last(query))

In [ ]:
ejex = query[:,"Day"] # Debemos transformar: Dates.format(query[2,"Date"], "yyyy-mm-dd HH:MM:SS")
ejey = query[:,"Meteocontrol Irrad (W/m2)"]

fig = plot(ejey,
     seriestype = :line, 
     title="Comportamiento Irradiancia", 
     xlabel="Tiempo", 
     ylabel="Irradiancia (W/m2)",
     xrotation=90,
     ylims=(0,1200),
     lab="Irradiancia",
     legend=:topright,
     ls=:dash,
     lc=:grey,
     m=:circle,
     ms=4,
     mc=:white)

In [ ]:
savefig(fig, "./irradiancia.png")